In [25]:
from ultralytics import YOLO
import cv2
import os
from datetime import datetime

model_name = 'last20epM.pt'

model = YOLO(model_name)

video_path = "1.mp4"

confidence_threshold = 0.7 # Set your confidence threshold

frame_max = 20 #amount of frames to analyze before start searching again

window_width = 640

folder_name_input = ''

#------------------------------------------------------

if folder_name_input:
    folder_name = folder_name_input
else:
    now = datetime.now()
    dt_string = now.strftime("%Y-%m-%d__%H-%M-%S")
    folder_name = f'{dt_string}__{video_path}__{model_name}'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)

if video_path.isdigit():  # Check if the input is an integer (camera index)
    video_path = int(video_path)

cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)

window_height = int(window_width*cap.get(cv2.CAP_PROP_FRAME_HEIGHT)/cap.get(cv2.CAP_PROP_FRAME_WIDTH))

cv2.namedWindow("YOLOv8 Inference", cv2.WINDOW_NORMAL)
cv2.resizeWindow("YOLOv8 Inference", window_width, window_height)

fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter(f'{folder_name}.mp4', fourcc, fps, (window_width, window_height))


frame_count = 0
frames = []
counter = 0
frames_counter =0
conf_array = []
conf_array.append(0)
conf_float = 0

#------------------------------------------------------

while True:
    _, img = cap.read()
    img_before = img.copy()
    results = model(img)
    
    annotated_frame = results[0].plot()

    for r in results:
        boxes = r.boxes
        for box in boxes:
            b = box.xyxy[0] 
            c = box.cls
            conf = box.conf
            conf_float = conf[0].cpu().numpy().item()
            conf_array.append(conf_float)

    if max(conf_array) > confidence_threshold or frame_max > frames_counter > 0:
        
        #cv2.putText(annotated_frame, f'recording frames', (330,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,255), 2)
        
        frames.append((img_before, annotated_frame, max(conf_array)))
        frames_counter += 1
        
    if frames_counter == frame_max:
        
        #cv2.putText(annotated_frame, 'frame_max', (30,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,255), 2)

        max_conf_frame = max(frames, key=lambda x: x[2])
        cv2.imwrite(os.path.join(folder_name, f'{counter:04}_1.png'), max_conf_frame[0])
        cv2.imwrite(os.path.join(folder_name, f'{counter:04}_2.png'), max_conf_frame[1])
        counter += 1
        frames = []
        conf_array = []
        conf_array.append(0)
        frames_counter = 0

    cv2.imshow('YOLOv8 Inference', annotated_frame)
    annotated_frame = cv2.resize(annotated_frame, (window_width, window_height))
    out.write(annotated_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 94.2ms
Speed: 0.0ms preprocess, 94.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 142.2ms
Speed: 0.0ms preprocess, 142.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 60.3ms
Speed: 0.0ms preprocess, 60.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.8ms
Speed: 0.0ms preprocess, 62.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 78.5ms
Speed: 0.0ms preprocess, 78.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 74.8ms
Speed: 0.0ms preprocess, 74.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 69.0ms
Speed: 0.0ms preprocess, 69.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 74.2ms
Speed: 2.0ms preprocess, 74.2ms


0: 384x640 (no detections), 71.5ms
Speed: 0.0ms preprocess, 71.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.7ms
Speed: 1.9ms preprocess, 63.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 69.6ms
Speed: 0.0ms preprocess, 69.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.0ms
Speed: 0.0ms preprocess, 62.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.9ms
Speed: 0.0ms preprocess, 65.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 57.3ms
Speed: 0.0ms preprocess, 57.3ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 60.5ms
Speed: 0.0ms preprocess, 60.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.8ms
Speed: 1.1ms preprocess, 66.8ms 


0: 384x640 (no detections), 68.3ms
Speed: 0.0ms preprocess, 68.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 57.9ms
Speed: 0.0ms preprocess, 57.9ms inference, 15.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 64.2ms
Speed: 0.0ms preprocess, 64.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 50.9ms
Speed: 2.3ms preprocess, 50.9ms inference, 15.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.8ms
Speed: 0.0ms preprocess, 61.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.0ms
Speed: 0.0ms preprocess, 65.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 64.1ms
Speed: 2.0ms preprocess, 64.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 71.3ms
Speed: 0.0ms preprocess, 71.3ms


0: 384x640 (no detections), 70.6ms
Speed: 0.0ms preprocess, 70.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.2ms
Speed: 2.0ms preprocess, 63.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 57.5ms
Speed: 0.0ms preprocess, 57.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.7ms
Speed: 1.1ms preprocess, 66.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.7ms
Speed: 2.1ms preprocess, 63.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.6ms
Speed: 0.0ms preprocess, 63.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.6ms
Speed: 1.1ms preprocess, 63.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.3ms
Speed: 0.0ms preprocess, 59.3ms i


0: 384x640 (no detections), 58.0ms
Speed: 0.0ms preprocess, 58.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 64.8ms
Speed: 0.0ms preprocess, 64.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 67.7ms
Speed: 2.0ms preprocess, 67.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 64.3ms
Speed: 0.0ms preprocess, 64.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.8ms
Speed: 2.9ms preprocess, 66.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 56.4ms
Speed: 0.0ms preprocess, 56.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.3ms
Speed: 1.8ms preprocess, 62.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 55.4ms
Speed: 0.0ms preprocess, 55.4ms i

...

Speed: 0.0ms preprocess, 56.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.4ms
Speed: 4.0ms preprocess, 63.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 55.9ms
Speed: 0.0ms preprocess, 55.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.9ms
Speed: 0.0ms preprocess, 66.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 56.5ms
Speed: 0.0ms preprocess, 56.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.0ms
Speed: 3.4ms preprocess, 65.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 54.7ms
Speed: 0.0ms preprocess, 54.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.9ms
Speed: 0.0ms preprocess, 59.9ms inference, 0.0ms postprocess per imag

Speed: 2.0ms preprocess, 64.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.0ms
Speed: 5.5ms preprocess, 65.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 54.7ms
Speed: 0.0ms preprocess, 54.7ms inference, 15.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 60.6ms
Speed: 0.0ms preprocess, 60.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.3ms
Speed: 0.0ms preprocess, 66.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 55.7ms
Speed: 0.0ms preprocess, 55.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.7ms
Speed: 0.0ms preprocess, 62.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.3ms
Speed: 3.1ms preprocess, 65.3ms inference, 0.0ms postprocess per ima


0: 384x640 (no detections), 65.0ms
Speed: 2.3ms preprocess, 65.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 67.9ms
Speed: 0.0ms preprocess, 67.9ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.1ms
Speed: 0.0ms preprocess, 61.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.1ms
Speed: 0.1ms preprocess, 65.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 55.3ms
Speed: 0.0ms preprocess, 55.3ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 60.0ms
Speed: 0.0ms preprocess, 60.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 dickghosts, 67.6ms
Speed: 0.0ms preprocess, 67.6ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 dickghosts, 60.7ms
Speed: 0.0ms preprocess, 60.7ms infere


0: 384x640 (no detections), 63.7ms
Speed: 2.0ms preprocess, 63.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.5ms
Speed: 2.0ms preprocess, 63.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.5ms
Speed: 3.3ms preprocess, 66.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 57.1ms
Speed: 0.0ms preprocess, 57.1ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.1ms
Speed: 0.0ms preprocess, 61.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.8ms
Speed: 2.0ms preprocess, 66.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.2ms
Speed: 2.4ms preprocess, 63.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 dickghosts, 56.6ms
Speed: 0.0ms preprocess, 56.6ms inf


0: 384x640 (no detections), 69.7ms
Speed: 0.0ms preprocess, 69.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.8ms
Speed: 2.2ms preprocess, 63.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 56.0ms
Speed: 0.0ms preprocess, 56.0ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 58.9ms
Speed: 0.0ms preprocess, 58.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.7ms
Speed: 0.0ms preprocess, 63.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.2ms
Speed: 3.3ms preprocess, 65.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 60.4ms
Speed: 0.0ms preprocess, 60.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.6ms
Speed: 0.0ms preprocess, 65.6ms 

In [19]:
print(fps)

25.0
